<a href="https://colab.research.google.com/github/kconstable/market_predictions/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing for Deep Learning


## Import Libraries 

In [260]:
import pandas as pd
import numpy as np
import math

import plotly.graph_objects as go

from keras.models import Sequential,backend
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data

In [261]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/data/market_data_full.pickle')
# df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/data/market_data_compact.pickle')

## Prepare Data for Deep Learning


### Prepare the Data 
+ Reverse the date-time index
+ Convert the date-time index to an integer index
+ Convert to numpay arrays

In [262]:
def prepare_data(df,y='close',features=[]):
  """
  Filter, scale and convert dataframe data to numpy arrays

  Inputs: 
    df       => A dataframe of observations with features and y-labels
    y        => The name of the column that is the truth labels
    features => A list of features.  Used to subset columns

  Outputs:
    scaled_y => numpy array of the y-label data
    scaled_x => numpy array of the training features

  """

  # reverse the index such that dates are in chronological order
  df = df.iloc[::-1]

  # Subset features, get the y-label values
  df_y = df[y]
  df_X = df[features]

  # save the date index, then replace the date index with an integer index
  df_X.reset_index(drop=True,inplace=True)

  # convert to numpay arrays
  array_X = np.array(df_X)
  array_y = np.array(df_y).reshape(-1,1)

  # print the output
  print("\nData Preparation")
  print("="*60)
  print(f"=> {len(features)} Features")
  print(f"=> Input Dimensions :{array_X.shape}")
  print(f"=> Output Dimensions:{array_y.shape}")
  print("\n")

  return array_y, array_X

### Split the Data into Train/Test Sets
+ Time series data cannot be split randomly like other observational data because the order is important.  
+ The data is split into train/test based on dates

In [263]:
def plot_train_test_data(train_x,test_x,idx_close):
  """
  """

  ts0 = list(range(0,train_x.shape[0]))
  ts1 = list(range(train_x.shape[0],train_x.shape[0]+test_x.shape[0]+1))

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=ts0,y=train_x[:,idx_close],name='Train'))
  fig.add_trace(go.Scatter(x=ts1,y=test_x[:,idx_close],name='Test'))
  fig.add_shape(type ='rect',
                  x0=ts1[0],x1=ts1[-1],
                  y0=0,y1=1,
                  line=dict(color='#F6B28D'),
                  fillcolor='#F6B28D',
                  opacity=0.1)
  fig.add_shape(type ='rect',
                  x0=ts0[0],x1=ts0[-1],
                  y0=0,y1=1,
                  line=dict(color='#7BA1AA'),
                  fillcolor='#7BA1AA',
                  opacity=0.1)
  fig.update_layout(title = 'Train/Test Datasets (Scaled)',
                      template="plotly_white",
                      yaxis_title='Closing Price (Scaled)',
                      xaxis_title='time steps',
                      width = 600,
                      height =600)
  fig.update_shapes(dict(xref='x',yref='paper'))
  fig.show()



In [264]:
def split_train_test(X,idx_close,train_pct=0.80):
  """
  """

  # Train
  train_num = math.ceil(X.shape[0]*train_pct)
  train_X = X[0:train_num,:]
  train_str = f'{round(train_X.shape[0]/X.shape[0],3)*100}%'

  # Test
  # test_X = X[train_num-n_steps:,:]
  test_X = X[train_num:,:]
  test_str =f'{round(test_X.shape[0]/X.shape[0],3)*100}%'

  # print the results
  print("\nSplit Data (X)")
  print("="*60)
  print(f'Data  :100.0% {X.shape}')
  print(f'Train :{train_str:>6} {train_X.shape}')
  print(f'Test  :{test_str:>6} {test_X.shape}')
  print('\n')

  # create the line plots
  plot_train_test_data(train_X,test_X,idx_close)

  return train_X,test_X


### Partition the Data

In [265]:
def create_partitions(data,idx_close,n_steps,n_predict,visualize=False):
  """
  This function partitions the train/test data into batches with times-step windows for training

  Each batch consists of n_steps of training data, and n_predict steps of label data
  The function outputs an x array [samples, time steps, features] and a y array [samples, time steps]

  Inputs:
    data      => train or test array
    idx_close => the position of y value in the data
    n_steps   => the number of time steps in each training batch
    n_predict => the number of time steps that will be predicted
    visualize => boolean, will plot a visual of training/prediction windows
  Outputs:
    array(i)  => np.array of batched & partitioned training data with features
    array(p)  => np.array of batched & partitioned y-lables 

  Reference:
    https://www.relataly.com/time-series-forecasting-multi-step-regression-using-neural-networks-with-multiple-outputs-in-python/5800/
  """
  n = data.shape[0]
  window = n_steps + n_predict
  i, p = [],[]

  # print the moving window
  if visualize:
    print("Data Window: I(Input), P(Predict),-(scanned), +(to be scanned)")
    print("="*max(n,100))

  # create the partitions
  for s in range(n_steps, n-n_predict):
    # get the input window + all features

    # print(s-n_steps,s+n_predict)
    i.append(data[s-n_steps:s,:])

    # get the prediction window + the closing price
    # print("X:",s-n_steps,s+n_predict,'\ty:',s,s+n_predict)
    p.append(data[s:s+n_predict,idx_close])

    # print the moving window
    if visualize and s <= 50:
      scanned = n-((s-n_steps)+window)
      print("-"*(s-n_steps),'I'*n_steps,'P'*n_predict,"+"*scanned,sep="")
    
  return np.array(i),np.array(p)

In [266]:
def plot_training_window(x_array,y_array,idx_close,n_steps,n_predict,batch):
  """
  Inputs: 
    x_array/y_array => partitioned data from create_partitions
    idx_close       => the position of the y-label (close price)
    n_steps         => the number of time steps in each training batch
    n_predict       => the number of time steps predicted each training batch
    batch           => the batch number to plot
  Outputs:
    A line chart with the train/predict values
  """
  # convert the arrays to dataframes
  # align the x indexes to compare
  df_y = pd.DataFrame(y_array[batch],index=range(n_steps-1,n_predict+n_steps-1),columns=['y'])
  df_x = pd.DataFrame(x_array[batch+1])[idx_close]
  df_x = pd.DataFrame(df_x)
  df_x.columns = ['x']

  # create the plots
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df_x.index,y=df_x['x'],name='train window'))
  fig.add_trace(go.Scatter(x=df_y.index,y=df_y['y'],name='predict window'))
  fig.update_layout(template='plotly_white',
                    title='Train/Predict Windows',
                    yaxis_title = 'Closing Price (Scaled)',
                    xaxis_title='Period')
  fig.show()


### Process the Data

In [267]:
# Training Config
# set seed
seed = 1985
np.random.seed(seed)

# Extract the features needed for training
features = [c for c in df.columns if c not in ['symbol','b-middle']]
n_steps  = 40   # The number of time steps included in each training batch
n_predict = 5   # The number of time steps into the future the model will predict
# idx_index = df.index  # date-time index of the df
idx_close = df.columns.get_loc("close") # index postiion variable (closing price)


# Scale the data and convert to numpy arrays
array_y,array_X = prepare_data(df,'close',features)


# scale the input and outputs
# scale to match the tanh activation function
scaler = MinMaxScaler(feature_range=(0,1))
scaled_X = scaler.fit_transform(array_X)
scaled_y = scaler.fit_transform(array_y)


# split into train, test
train_pct = 0.80
train_x,test_x=split_train_test(scaled_X,idx_close,train_pct)


# Partition the train/test data into time series windows for training
# LSTM input format: [samples, time steps, features]
x_train, y_train = create_partitions(train_x,idx_close,n_steps, n_predict)
x_test,  y_test  = create_partitions(test_x, idx_close,n_steps, n_predict)


# Print the results
print('Train/Test Dimensions')
print('='*60)
print("Train Data Dimensions: ","x",x_train.shape,"y",y_train.shape)
print("Test Data Dimensions : ","x",x_test.shape," y",y_test.shape)

print("\n\nCheck the dimensions of the training windows")
print("="*60)
print('The last value in the training batch should match the first value in the predict batch')
print("Last Training Value:   ",x_train[1][n_steps-1][idx_close])
print("First Prediction Value:",y_train[0][0])

# Make sure the train/predict batch windows are aligned
plot_training_window(x_train,y_train,idx_close,n_steps,n_predict,seed//10)

# save to google drive
np.save('/content/drive/MyDrive/Colab Notebooks/data/x_train',x_train)
np.save('/content/drive/MyDrive/Colab Notebooks/data/y_train',y_train)
np.save('/content/drive/MyDrive/Colab Notebooks/data/x_test',x_test)
np.save('/content/drive/MyDrive/Colab Notebooks/data/y_test',y_test)



Data Preparation
=> 23 Features
=> Input Dimensions :(2484, 23)
=> Output Dimensions:(2484, 1)



Split Data (X)
Data  :100.0% (2484, 23)
Train : 80.0% (1988, 23)
Test  : 20.0% (496, 23)




Train/Test Dimensions
Train Data Dimensions:  x (1943, 40, 23) y (1943, 5)
Test Data Dimensions :  x (451, 40, 23)  y (451, 5)


Check the dimensions of the training windows
The last value in the training batch should match the first value in the predict batch
Last Training Value:    0.020153164046755337
First Prediction Value: 0.020153164046755337


In [10]:
# np.load('/content/drive/MyDrive/Colab Notebooks/data/y_test.npy')

# Deep Learning LSTM

### Build the LSTM Model

In [268]:
def create_lstm_model(config,x_train):
  """
  """
  # get window size
  n_steps = config['data']['n_steps']
  n_predict = config['data']['n_predict']
  n_features = x_train.shape[2]

  # clear previous models
  backend.clear_session()

  # LSTM Model+ first layer
  model = Sequential(name='LSTM')
  model.add(LSTM(n_steps,return_sequences=True,input_shape=(n_steps,n_features)))

  # add additional layers
  for layer,nodes,ret_seq,drop in config['layers']:
    # add LSTM layers + dropout
    if layer=='lstm':
      model.add(LSTM(nodes,return_sequences =ret_seq))
      if drop is not None:
        model.add(Dropout(drop))

    # Add Dense Layers + dropout
    elif layer =='dense':
      model.add(Dense(nodes))
      if drop is not None:
        model.add(Dropout(drop))

  # add the prediction layer
  model.add(Dense(n_predict))
  
  # compile
  model.compile(optimizer='adam',loss='mse')
  model.summary()

  return model


### Calculate Model Performance

In [269]:
def calculate_performance(x_test,y_test,model,scaler):
  """
  """

  # Predict the prices
  y_pred = model.predict(x_test)

  # convert units back to the original scale
  y_pred_unscaled = scaler.inverse_transform(y_pred)
  y_test_unscaled = scaler.inverse_transform(y_test)

  # Root mean squarred error,mean abs error, mean abs percent error, median abs percent error
  rmse  = math.sqrt(mean_squared_error(y_test_unscaled, y_pred_unscaled))
  mae   = mean_absolute_error(y_test_unscaled, y_pred_unscaled)
  mape  = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred_unscaled)/ y_test_unscaled))) * 100
  mdape = np.median((np.abs(np.subtract(y_test_unscaled, y_pred_unscaled)/ y_test_unscaled)) ) * 100

  print("\nModel Error")
  print("="*62)
  print(f'{"Mean Absolute Error (MAE)" :-<55} {np.round(mae, 2):>5}')
  print(f'{"Root Mean Squared Error (MSE)" :-<55} {np.round(rmse,2):>5}')
  print(f'{"Mean Absolute Percentage Error (MAPE)" :-<55} {np.round(mape, 2):>5}%')
  print(f'{"Median Absolute Percentage Error (MDAPE)" :-<55} {np.round(mdape, 2):>5}%')


### Plot Training Metrics

In [270]:

def plot_training_metrics(history):
  """
  """
  # get the number of epochs
  epochs = list(range(1, len(history.history['loss']) + 1))

  min_train_loss = min(history.history['loss'])
  min_val_loss = min(history.history['val_loss'])

  # create the line plots
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=epochs,y=history.history['loss'],
                           name = 'train-loss',
                           line=dict(width=3,color='royalblue')))
  fig.add_trace(go.Scatter(x=epochs,y=history.history['val_loss'],
                           name='val-loss',
                           line=dict(width=3,color='crimson')))

  fig.add_shape(type ='rect',
                x0=0,
                x1=1,
                y0=min(min_train_loss,min_val_loss),
                y1=max(min_train_loss,min_val_loss),
                line=dict(color='#ccc'),
                fillcolor='#ccc',
                opacity=0.4)
  fig.update_layout(title = 'Training Metrics',template="plotly_white",yaxis_title='loss',xaxis_title='epochs')
  fig.update_shapes(dict(xref='paper',yref='y'))
  fig.show()

### Train the Model

In [271]:
config ={
    'data': {'n_steps':n_steps,'n_predict':n_predict},
    'layers': [('lstm',128,True,0.2),('lstm',64,False,0.2),('dense',64,None,0.3),('dense',16,None,0.1)],
    'train':{'epochs':100,'batch_size':128,'early_stop':EarlyStopping(monitor='loss',patience=10)}
}
config

{'data': {'n_predict': 5, 'n_steps': 40},
 'layers': [('lstm', 128, True, 0.2),
  ('lstm', 64, False, 0.2),
  ('dense', 64, None, 0.3),
  ('dense', 16, None, 0.1)],
 'train': {'batch_size': 128,
  'early_stop': <keras.callbacks.EarlyStopping at 0x7f439f273f50>,
  'epochs': 100}}

In [272]:
np.random.seed(seed)
model = create_lstm_model(config,x_train)

history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10,
                    verbose = 0
                    )

calculate_performance(x_test,y_test,model,scaler)
plot_training_metrics(history)


Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            10240     
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 128)           86528     
_________________________________________________________________
dropout (Dropout)            (None, 40, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0      

In [337]:

# Plot train+test+predict
# test numbers don't line up 
#=> check create paritions to see why the number of records don't match


# train data
train_unscaled = scaler.inverse_transform(train_x)
df_train=pd.DataFrame(train_unscaled[:,idx_close])
df_train.columns=['price']
print(df_train.shape)


# test data
test_unscaled = scaler.inverse_transform(test_x)
df_test = pd.DataFrame(test_unscaled[:,idx_close])
df_test.columns = ['price']


# reindex the the test set to start from the train set
idx_last_train = df_train.shape[0]
idx_last_test = df_test.shape[0]
idx_rng = list(range(idx_last_train,idx_last_train + idx_last_test))

df_test['idx'] = idx_rng
df_test.set_index('idx',drop=True,inplace=True)
print(df_test.shape)



# get the predictions
y_pred = model.predict(x_test)

# convert units back to the original scale
y_pred_unscaled = scaler.inverse_transform(y_pred)

# reset the index to align with the train/test indexes
# idx will be supplied by the caller function
idx = y_test_unscaled.shape[0]-1
idx_pred_start = idx_last_train  + idx_last_test -(idx_last_test - idx)
idx_rng = list(range(idx_pred_start,idx_pred_start+n_predict))



y_pred =y_pred_unscaled[idx,:]
df_pred = pd.DataFrame(y_pred)
df_pred.columns =['price']
df_pred['idx']=idx_rng
df_pred.set_index('idx',drop=True,inplace=True)
print(df_pred.shape)


fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train.index,y=df_train.price,name='Train'))
fig.add_trace(go.Scatter(x=df_test.index,y=df_test.price,name='Test'))
fig.add_trace(go.Scatter(x=df_pred.index,y=df_pred.price,name='Predict'))
fig.show()  

print(idx,idx_last_train,idx_last_test,idx_pred_start)
df_pred



(1988, 1)
(496, 1)
(5, 1)


450 1988 496 2438


,price
idx,
2438,28.688049
2439,28.799809
2440,28.616255
2441,28.703285
2442,28.760162
